In [1]:
# NOTE: change this to the path in your setup
korsmit_exp1_path = "../../data/Korsmit/Exp1/"

In [2]:
import os
import pandas as pd
import numpy as np

## Load csv files and extract related columns

In [6]:
IDim_path = korsmit_exp1_path+"Data/IDim/"
IDim_response_dfs = []

for file in sorted(os.listdir(IDim_path)):
    if file.endswith(".csv"):
        file_path = os.path.join(IDim_path, file)
        try:
            df = pd.read_csv(file_path, sep=r'\s*,\s*', engine='python')
            # Crucial: Strip whitespace from column names
            df.columns = df.columns.str.strip()

            # Ensure required rating columns exist
            required_cols = ['positive', 'relaxed', 'awake', 'like']
            if all(col in df.columns for col in required_cols):
                # Select only the relevant columns and append to our list
                IDim_response_dfs.append(df[required_cols])
            else:
                print(f"Skipping file '{file_path}': Missing required columns ({required_cols}). Found columns: {df.columns.tolist()}")

        except Exception as e:
            print(f"Error reading or processing file {file_path}: {e}")


# Concatenate all individual DataFrames into one master DataFrame for human responses
if IDim_response_dfs:
    master_human_responses_df = pd.concat(IDim_response_dfs, ignore_index=True)
    print(f"Master human responses DataFrame shape: {master_human_responses_df.shape}\n")
    print(f"Master human responses (first 5 rows):\n{master_human_responses_df}\n")
else:
    raise ValueError("No valid CSV files found or processed in IDim_path.")

Master human responses DataFrame shape: (3835, 4)

Master human responses (first 5 rows):
      positive  relaxed  awake  like
0         3.68     3.78   4.42  3.41
1         5.88     5.98   3.89  5.54
2         6.53     5.59   6.59  6.17
3         6.26     5.71   6.88  6.18
4         2.80     2.62   5.15  1.87
...        ...      ...    ...   ...
3830      7.04     4.09   6.18  6.41
3831      7.56     5.32   5.54  7.90
3832      6.01     2.73   7.99  5.57
3833      2.04     1.49   9.00  3.03
3834      4.78     3.70   7.67  6.24

[3835 rows x 4 columns]



# Extract precomputed timbre features

In [4]:
# Path to your timbre features CSV
timbre_csv_path = korsmit_exp1_path+'Data/long_AT1_TimbreToolbox_220509.csv'

# Load CSV
timbre_df = pd.read_csv(timbre_csv_path)

# Strip any extra whitespace from column names (good practice)
timbre_df.columns = timbre_df.columns.str.strip()
timbre_df.columns

Index(['ppno', 'perceived/induced', 'dimensional/discrete', 'sequence',
       'stimNo', 'stim',
       'family (B = brass; W = woodwind; S = string; P = percussion)', 'instr',
       'octave', 'nPlays', 'like', 'positive', 'relaxed', 'tense', 'awake',
       'anger', 'fear', 'sadness', 'happiness', 'tenderness', 'IQR_Pitch',
       'IQR_HarmonicSpectralDeviation', 'IQR_Tristimulus_1',
       'IQR_Tristimulus_2', 'IQR_Tristimulus_3', 'IQR_HarmonicOddToEvenRatio',
       'IQR_Inharmonicity', 'IQR_HarmonicEnergy', 'IQR_NoiseEnergy',
       'IQR_Noisiness', 'IQR_HarmonicToNoiseEnergy',
       'IQR_PartialsToNoiseEnergy', 'Median_Pitch',
       'Median_HarmonicSpectralDeviation', 'Median_Tristimulus_1',
       'Median_Tristimulus_2', 'Median_Tristimulus_3',
       'Median_HarmonicOddToEvenRatio', 'Median_Inharmonicity',
       'Median_HarmonicEnergy', 'Median_NoiseEnergy', 'Median_Noisiness',
       'Median_HarmonicToNoiseEnergy', 'Median_PartialsToNoiseEnergy',
       'IQR_SpectralCentroi

In [7]:
feature_names = [
    'IQR_Pitch','IQR_HarmonicSpectralDeviation', 'IQR_Tristimulus_1',
    'IQR_Tristimulus_2', 'IQR_Tristimulus_3', 'IQR_HarmonicOddToEvenRatio',
    'IQR_Inharmonicity', 'IQR_HarmonicEnergy', 'IQR_NoiseEnergy',
    'IQR_Noisiness', 'IQR_HarmonicToNoiseEnergy',
    'IQR_PartialsToNoiseEnergy', 'Median_Pitch',
    'Median_HarmonicSpectralDeviation', 'Median_Tristimulus_1',
    'Median_Tristimulus_2', 'Median_Tristimulus_3',
    'Median_HarmonicOddToEvenRatio', 'Median_Inharmonicity',
    'Median_HarmonicEnergy', 'Median_NoiseEnergy', 'Median_Noisiness',
    'Median_HarmonicToNoiseEnergy', 'Median_PartialsToNoiseEnergy',
    'IQR_SpectralCentroid', 'IQR_SpectralSpread', 'IQR_SpectralSkewness',
    'IQR_SpectralKurtosis', 'IQR_SpectralFlatness', 'IQR_SpectralCrest',
    'IQR_SpectralSlope', 'IQR_SpectralDecrease', 'IQR_SpectralRollOff',
    'IQR_SpectralVariation', 'IQR_SpectralFlux', 'Median_SpectralCentroid',
    'Median_SpectralSpread', 'Median_SpectralSkewness',
    'Median_SpectralKurtosis', 'Median_SpectralFlatness',
    'Median_SpectralCrest', 'Median_SpectralSlope',
    'Median_SpectralDecrease', 'Median_SpectralRollOff',
    'Median_SpectralVariation', 'Median_SpectralFlux', 'AttackTime',
    'LogAttackTime', 'AttackSlope', 'DecreaseSlope', 'TemporalCentroid',
    'EffectiveDuration', 'FrequencyOfEnergyModulation',
    'AmplitudeOfEnergyModulation'
]

# Limit timbre_df to the first 59 rows (59 sounds)
X_base = timbre_df.loc[:58].sort_values('stimNo')[feature_names].values
  # shape: (59, num_features)

# Repeat each sound's features for all participants
# Assume each participant rated all 59 sounds
num_participants = int(master_human_responses_df.shape[0] / 59)

X = np.tile(X_base, (num_participants, 1))  # shape: (3835, num_features)

mask = np.isnan(X)

X = np.where(mask, 0, X)

y = master_human_responses_df[['positive', 'relaxed', 'awake', 'like']].values

# Sanity check
assert X.shape[0] == y.shape[0], "Mismatch between feature and label rows!"


# Prepare features X and targets y

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# --- Split Data into Training and Testing Sets ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42
)

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(f"Training set size (X_train, y_train): {X_train.shape}, {y_train.shape}")
print(f"Testing set size (X_test, y_test): {X_test.shape}, {y_test.shape}\n")

Training set size (X_train, y_train): (3451, 54), (3451, 4)
Testing set size (X_test, y_test): (384, 54), (384, 4)



# Train regression head (=MLP, a few projection layers)

In [9]:
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor

mlp_regressor = MLPRegressor(
    hidden_layer_sizes=(100, 50),
    activation='relu',
    solver='adam',
    max_iter=1000,
    random_state=42,
    verbose=True,
    early_stopping=True,
    n_iter_no_change=50,
    tol=1e-4
)

# rf = RandomForestRegressor(n_estimators=5000)

print("Starting MLP Regressor training...")
mlp_regressor.fit(X_train, y_train)
print("\nMLP Regressor training complete.")

Starting MLP Regressor training...
Iteration 1, loss = 11.49583762
Validation score: -3.030509
Iteration 2, loss = 5.33952794
Validation score: -0.418710
Iteration 3, loss = 2.21825406
Validation score: 0.051652
Iteration 4, loss = 1.77615158
Validation score: 0.155968
Iteration 5, loss = 1.64255942
Validation score: 0.201188
Iteration 6, loss = 1.58921432
Validation score: 0.218646
Iteration 7, loss = 1.55623041
Validation score: 0.230294
Iteration 8, loss = 1.53539996
Validation score: 0.235867
Iteration 9, loss = 1.52523780
Validation score: 0.242084
Iteration 10, loss = 1.51357013
Validation score: 0.244314
Iteration 11, loss = 1.50619342
Validation score: 0.248097
Iteration 12, loss = 1.50127935
Validation score: 0.248173
Iteration 13, loss = 1.49555640
Validation score: 0.251828
Iteration 14, loss = 1.49252046
Validation score: 0.251540
Iteration 15, loss = 1.49128183
Validation score: 0.250392
Iteration 16, loss = 1.49449312
Validation score: 0.250979
Iteration 17, loss = 1.4896

# Evaluate

In [10]:
y_test

array([[3.96, 8.32, 7.05, 8.  ],
       [7.11, 7.01, 8.11, 7.01],
       [4.68, 5.04, 5.88, 5.1 ],
       ...,
       [1.04, 1.  , 7.02, 1.98],
       [8.72, 8.88, 8.84, 8.75],
       [6.69, 4.62, 7.13, 6.38]])

In [15]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr

y_pred = mlp_regressor.predict(X_test)

print(f"\nShape of predictions (y_pred): {y_pred.shape}")
print(f"First 5 actual values (y_test):\n{y_test[:5]}")
print(f"First 5 predicted values (y_pred):\n{y_pred[:5]}\n")

# Evaluation Metrics:

# Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae:.4f}")

# Mean Absolute Percentage Error (MAPE)
absolute_percentage_error = np.abs((y_test - y_pred) / y_test) * 100
mape = np.mean(absolute_percentage_error)

print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%\n")

# Root Mean Squared Error (RMSE)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")

# Pearson Correlation Coefficient (per dimension)
print("\nPearson Correlation Coefficients (per dimension):")
for i, dim_name in enumerate(['positive', 'relaxed', 'awake', 'like']):
    # Check for sufficient variance to calculate correlation
    if np.std(y_test[:, i]) > 1e-6 and np.std(y_pred[:, i]) > 1e-6:
        correlation, _ = pearsonr(y_test[:, i], y_pred[:, i])
        print(f"  {dim_name} Dimension: {correlation:.4f}")
    else:
        print(f"  {dim_name} Dimension: Cannot calculate (insufficient variance in data for this dimension)")

correlations = []
for i in range(y_test.shape[1]):
    if np.std(y_test[:, i]) > 1e-6 and np.std(y_pred[:, i]) > 1e-6:
        correlations.append(pearsonr(y_test[:, i], y_pred[:, i])[0])
if correlations:
    average_correlation = np.mean(correlations)
    print(f"Average Pearson Correlation across dimensions: {average_correlation:.4f}")
else:
    print("  No correlations could be calculated for averaging.")


from sklearn.metrics import r2_score

# R-squared
print("\nR-squared scores:")
r2_valence = r2_score(y_test[:, 0], y_pred[:, 0])
print("  valence =", r2_valence)

r2_tension = r2_score(y_test[:, 1], y_pred[:, 1])
print("  tension =", r2_tension)

r2_energy = r2_score(y_test[:, 2], y_pred[:, 2])
print("  energy =", r2_energy)

r2_like = r2_score(y_test[:, 3], y_pred[:, 3])
print("  like =", r2_like)

avg_r2 = r2_score(y_test, y_pred)
print("Average R-squared =", r2_like)



Shape of predictions (y_pred): (384, 4)
First 5 actual values (y_test):
[[3.96 8.32 7.05 8.  ]
 [7.11 7.01 8.11 7.01]
 [4.68 5.04 5.88 5.1 ]
 [2.08 4.04 7.6  1.26]
 [6.34 6.43 5.32 4.8 ]]
First 5 predicted values (y_pred):
[[5.38915467 5.86170046 5.21601801 5.40391586]
 [4.69755393 4.96583829 4.85676618 4.91104396]
 [3.8179784  3.19733877 6.79529714 3.3575092 ]
 [2.81831699 2.20345215 6.95396629 2.25166629]
 [5.11046045 4.81583446 6.30417302 4.68668885]]

Mean Absolute Error (MAE): 1.3833
Mean Absolute Percentage Error (MAPE): 41.12%

Root Mean Squared Error (RMSE): 1.7195

Pearson Correlation Coefficients (per dimension):
  positive Dimension: 0.5701
  relaxed Dimension: 0.6171
  awake Dimension: 0.3194
  like Dimension: 0.5854
Average Pearson Correlation across dimensions: 0.5230

R-squared scores:
  valence = 0.32209374473261776
  tension = 0.37903183716006694
  energy = 0.10141364495065741
  like = 0.3415332670729845
Average R-squared = 0.3415332670729845
